In [1]:
from scipy.stats import truncnorm
import pandas as pd
import numpy as np
import itertools
import datetime
import tqdm
import sys
import os

def flatten_list(list_array):
    return list(itertools.chain(*list_array))

sys.path.insert(0,"../")
from global_config import config

results_dir           = config.get_property('results_dir')
data_dir              = config.get_property('data_dir')
paper_dir             = config.get_property('paper_dir')
data_db_dir           = config.get_property('data_db_dir')
feb_hosp_records_path = os.path.join(data_db_dir, 'long_files_8_25_2021')
path_to_save          = os.path.join(results_dir, "real_testing", "community")

COLOR_LIST1           = ["#F8AFA8", "#FDDDA0", "#F5CDB4", "#74A089"]

In [2]:
adht_ward_df = pd.read_csv(os.path.join(data_db_dir, "long_files_8_25_2021", "counts_ward.csv" ), parse_dates=['date'])

date_start = pd.to_datetime('2020-02-01')
date_end   = pd.to_datetime('2021-02-28')

dates_sim  = pd.date_range(date_start, date_end)

adht_ward_df   = adht_ward_df[adht_ward_df.date.isin(dates_sim)]
#selected_ward = ['Allen Hospital', 'Harkness Pavilion', 'Milstein Hospital', 'Mschony', 'Presbyterian Hospital']

A_df     = pd.pivot(adht_ward_df, index='ward', columns='date', values='num_admitted')
D_df     = pd.pivot(adht_ward_df, index='ward', columns='date', values='num_discharged')
H_df     = pd.pivot(adht_ward_df, index='ward', columns='date', values='num_hospitalized')
tests_df = pd.pivot(adht_ward_df, index='ward', columns='date', values='num_tested')

pop        = H_df.mean(axis=1)
num_pop    = len(pop)
ward_names = pop.index

ward_num          = len(ward_names)
ward_transfers_df = pd.read_csv(os.path.join(data_db_dir, "long_files_8_25_2021", "transfers_ward.csv"), parse_dates=['date'])
ward_transfers_df = ward_transfers_df[ward_transfers_df.date.isin(dates_sim)]

M_df = np.zeros((ward_num, ward_num, len(dates_sim)+1))

for i in range(ward_num):
    ward_from = ward_names[i]
    for j in range(ward_num):
        ward_to      = ward_names[j]
        transfers_ij = ward_transfers_df[(ward_transfers_df.ward_from==ward_from) & (ward_transfers_df.ward_to==ward_to)]

        if(transfers_ij.shape[0] > 0) :
            dates_ij                = transfers_ij.date.values
            dates_ind               = np.where(np.in1d(dates_ij, dates_sim))[0]
            transfered              = transfers_ij.num_transfered.values
            M_df[i, j, dates_ind-1] = transfered

In [35]:
from models import process_metapop, observe_metapop, init_metapop, simulate

if_settings = {
   "Nif"                : 50,          # number of iterations of the IF
   "type_cooling"       : "geometric", # type of cooling schedule
   "shrinkage_factor"   : 0.9,         # shrinkage factor for the cooling schedule
   "inflation"          : 1.01,        # inflation factor for spreading the variance after the EAKF step
}

model_settings = {
    "param_name"  : ["ρ", "β"],   # importation and transmission rate
    "p"           : 2,              # number of parameters
    "k"           : num_pop,        # number of observations | We are just observing carriage
    "n"           : 3*num_pop,      # number of state variables / dimension of the state space
    "dt"          : 1,              # time step
    "T"           : len(dates_sim), # time to run
    "m"           : 300,           # number of ensembles
    "stochastic"  : True,           # is stochastic
    "num_pop"     : num_pop,
    "dates"       : dates_sim
    }

p = model_settings["p"]
m = model_settings["m"]
T = model_settings["T"]

delta = 1/120  # decolonization rate

A = A_df.to_numpy()
D = D_df.to_numpy()
H = H_df.to_numpy()
M = M_df

#tests = tests_df.to_numpy()
tests = np.zeros((num_pop, T))
tests = tests_df.to_numpy()

# Process model for the ifeakf | model(x, gamma, beta, delta, rho, sigma, pop, m=1, stochastic=True)
process_model_gamma = lambda t, x, θ, gamma : process_metapop(t, x,
                                        gamma          = [gamma] * model_settings["m"],
                                        beta           = θ[1, :],
                                        delta          = delta,
                                        Nmean          = pop,
                                        N              = H[:,  t],
                                        A              = A[:,  t],
                                        D              = D[:,  t],
                                        M              = M[:,:,t],
                                        model_settings = model_settings)

# Observational model for the ifeakf |  g(t, x, rho)
observational_model  = lambda t, x, θ: observe_metapop(
                                        t,
                                        x,
                                        N              = H[:, t],
                                        rho            = θ[0, :],
                                        tests          = tests[:,t],
                                        model_settings = model_settings)

# f0 model for the ifeakf            | initial_condition(c0, pop=2000, m=300)
initial_guess_x0_gamma  = lambda θ, gamma:  init_metapop(
                                        N0             = H[:, 0],                      # population size
                                        c0             = [gamma]*model_settings["m"], # importation rate
                                        model_settings = model_settings)


ρ_min = 0.01  # test sensitivity minimum
ρ_max = 0.5  # test sensitivity maximum

βmin = 0.00  # transmission rate minimum
βmax = 0.5   # transmission rate maximum

max_total_pop     = np.max(H.sum(axis=0))
state_space_range = np.array([0, max_total_pop])
parameters_range  = np.array([[ρ_min, ρ_max],
                              [βmin, βmax]])

σ_perturb         = np.array([(ρ_max-ρ_min)/4, (βmax-βmin)/4])


In [36]:
def amro2cute(amro):
    if amro == 'ESCHERICHIA COLI':
        return "e_coli"
    elif amro == 'KLEBSIELLA PNEUMONIAE':
        return "k_pneumoniae"
    elif amro=="PSEUDOMONAS AERUGINOSA":
        return "p_aeruginosa"
    elif amro=="METHICILLIN-SUSCEPTIBLE STAPHYLOCOCCUS AUREUS":
        return "mssa"
    elif amro=="METHICILLIN-RESISTANT STAPHYLOCOCCUS AUREUS":
        return "mrsa"
    elif amro=="STAPHYLOCOCCUS EPIDERMIDIS":
        return "s_epidermidis"
    elif amro=="ENTEROCOCCUS FAECALIS":
        return "e_faecalis"
    elif amro=="ENTEROCOCCUS FAECIUM":
        return "e_faecium"


In [37]:
import sys
sys.path.insert(0, "../pompjax/pompjax/")

from pyro.contrib.forecast import eval_crps
from eval import calibration


In [38]:
def compute_evals(samples, obs, beta, rho,  name_var="beta"):
    """_summary_

    Args:
        samples: num_ensembles x num_times
        obs:     time series observation

    Returns:
        pd.DataFrame: Dataframe with the continuos ranked probability score (crps) and the average calibration.
    """

    cal_df = calibration.calibration(np.expand_dims(samples.T, 0), np.expand_dims(obs, 0), observation_index=0)
    sc     = np.mean(np.abs(cal_df.quantiles.values-cal_df.proportion_inside.values))

    df_response                      = pd.DataFrame(columns=['crps', 'calibration_score', name_var, "rho"])
    df_response['crps']              = [eval_crps(samples, obs)]
    df_response["calibration_score"] = sc
    df_response[name_var]            = [beta]
    df_response['rho']               = [rho]

    return df_response


In [41]:
ρ_min    = 1/100
ρ_max    = 20/100

βmin     = 0.01
βmax     = 0.5

ρ_search = np.linspace(ρ_min, ρ_max, 100)
β_search = np.linspace(βmin, βmax, 100)

ρsim     = ρ_search[0]
βsim     = β_search[0]

θsim = np.array([[ρsim], [βsim]]) * np.ones((2, model_settings["m"]))

amro_search = ['ESCHERICHIA COLI', 'KLEBSIELLA PNEUMONIAE', 'PSEUDOMONAS AERUGINOSA', 'METHICILLIN-SUSCEPTIBLE STAPHYLOCOCCUS AUREUS',
                'METHICILLIN-RESISTANT STAPHYLOCOCCUS AUREUS', 'STAPHYLOCOCCUS EPIDERMIDIS', 'ENTEROCOCCUS FAECALIS', 'ENTEROCOCCUS FAECIUM']

amro         = amro_search[0]
amro_prev_df = pd.read_csv(os.path.join("..", "data", "amro_prevalence.csv"))
γ            = amro_prev_df[amro_prev_df.amro==amro]["prevalence_mean1"].values[0]/100

process_model    = lambda t, x, θ : process_model_gamma(t, x, θ, gamma=γ)
initial_guess_x0 = lambda θ:  initial_guess_x0_gamma(θ, gamma=γ)

def simulate(model, observe, initial_x0, θsim, model_settings):
    """ Simulate model with initial conditions and parameters

    Args:
        model (function):        Process model
        observe (function):      Observational model
        initial_x0 (function):   Initial condition guess model.
        θ_sim (np.array):        Parameters
    """

    n = model_settings["n"]
    k = model_settings["k"]
    m = model_settings["m"]
    T = model_settings["T"]

    x0 = initial_x0(θsim)
    if(x0.shape[0] != n or x0.shape[1] != m) :
        print('error in x0 dimensions')

    x_sim = np.full((n, m, T), np.nan)
    y_sim = np.full((k, m, T), np.nan)

    x_sim[:, :, 0] = x0
    y_sim[:, :, 0] = observe(0, x0, θsim)

    for t in tqdm.tqdm(range(1, T-1)):
        x_sim[:, :, t] = model(t, x_sim[:, :, t-1], θsim)
        y_sim[:, :, t] = observe(t, x_sim[:, :, t], θsim)

    return x_sim, y_sim

_, y_sim = simulate(process_model, observational_model, initial_guess_x0, θsim, model_settings)

100%|█████████▉| 392/393 [21:25:05<03:16, 196.70s/it]     


IndexError: index 393 is out of bounds for axis 1 with size 393

In [ ]:
y_sim.shape

In [45]:
A.shape

(219, 393)